# Importance 11 Subdistricts: thana_process

### Importance11 Subdistricts: thana_process
- DONE: Step 1: List all files and check if empty. Subset based on emptyness
- DONE: Step 2: Read file and add file name, split file name to additional columns and add column with path
- DONE: Step 3: Define variables and if necessary to calculate relative values
- DONE: Step 4: Determine how to sumarize columns if possible to sum or average and do so
- DONE: Step 5: Break data by years
- DONE: Step 6: Summarize data by zilas. Keep numeric variables, division, zila and year
- DONE: Step 7: Create file with zilas, division and 
- DONE: Step 8: Write everything to the output files

- Variables:
    - **thana**:
    - **ECouple**: 
    - **Pill**:
    - **Condom**:
    - **Injectable**: 
    - **IUD**: 
    - **Implant**:
    - **PerMale**: 
    - **PerFemale**:
    - **Tpermanent**: 
    - **GrandTotal**:
    - **CAR**:
    - **Unnamed: 12**:
    - **full_name_path**: 
    - **file_name1**: 
    - **file_name2**:
    - **year**: 
    - **month**: 
    - **upazila**: 
    - **division**: 
    - **upazila_full**:
- Indicators:
    - Percent_Pill = Pill/GrandTotal
    - Percent_Condom = Condom/GrandTotal
    - Percent_Injectable = Injectable/GrandTotal
    - Percent_IUD = IUD/GrandTotal
    - Percent_Implant = Implant/GrandTotal
    - Percent_PerMale = PerMale/Tpermanent
    - Percent_PerFemale = PerFemale/Tpermanent
    - CAR

## Paths

In [6]:
DATA = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/dgfp/data/importance11_subdistricts/thana_process'
GEO = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/dgfp/geos/dgfp_geo.csv'
OUT = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/dgfp/data/'

## Packages

In [7]:
import os
import re
import sys
import glob
import numpy as np
import pandas as pd

## Custom functions

In [8]:
def list_files(path):
    files_list = []
    for path, subdirs, files in os.walk(path):
        for file in files:
            if bool(re.search(pattern=r'.csv', string=file)):
                files_list.append(os.path.join(path, file))
    return files_list
    
def read_arrange_files(files_list):
    data_list = {}
    log_columns = ['empty', 'full_name_path', 'file_name1', 'file_name2',
                   'year', 'month', 'upazila', 'division', 'upazila_full']
    log_array = []
    for file in files_list:
        tmp = pd.read_csv(file, sep='\t', skiprows=1)
        log_list = []
        log_list.append(tmp.empty)
        if tmp.empty:
            log_list.append(file)
            log_list.append(os.path.split(file)[1])
            name = os.path.split(file)[1]
            name = name.replace(".csv", "")
            log_list.append(name)
            name_split = name.split("-")
            log_list.append(name_split[0])
            log_list.append(name_split[1])
            log_list.append(name_split[3])
            log_list.append(name_split[4])
            log_list.append(name_split[4]+name_split[3])
        else:
            tmp['full_name_path'] = file
            log_list.append(file)
            tmp['file_name1'] = os.path.split(file)[1]
            log_list.append(os.path.split(file)[1])
            name = os.path.split(file)[1]
            name = name.replace(".csv", "")
            tmp['file_name2'] = name
            log_list.append(name)
            name_split = name.split("-")
            tmp['year'] = int(name_split[0])
            log_list.append(name_split[0])
            tmp['month'] = int(name_split[1])
            log_list.append(name_split[1])
            tmp['upazila'] = name_split[3]
            log_list.append(name_split[3])
            tmp['division'] = name_split[4]
            log_list.append(name_split[4])
            tmp['upazila_full'] = name_split[4]+name_split[3]
            log_list.append(name_split[4]+name_split[3])
            data_list[name] = tmp
        log_array.append(log_list)
    return data_list, pd.DataFrame(log_array, columns=log_columns)

def concat_data(ddict):
    out_list = []
    for key, value in ddict.items():
        out_list.append(value)
    df = pd.concat(out_list, axis=0)
    return df

def break_by_years(input_df, year_var):
    df = input_df.copy(deep=True)
    out_dict = {}
    for year in df[year_var].unique():
        out_dict[str(year)] = df[df[year_var] == year]
    return out_dict

def summarize_zilas(data_dict):
    out_dict = {}
    for key, value in data_dict.items():
        tmp = value.copy(deep=True)
        subset_vars = ['upazila', 'division', 'ECouple', 'Pill', 'Condom', 'Injectable', 'IUD',
                       'Implant','PerMale', 'PerFemale', 'Tpermanent', 'GrandTotal']
        tmp = tmp[subset_vars]
        tmp = tmp.groupby('upazila').sum()
        tmp = tmp.reset_index()
        out_dict[key] = tmp
    return out_dict


def calculate_indicators(data_dict):
    out_dict = {}
    for key, value in data_dict.items():
        tmp = value.copy(deep=True)
        tmp['Percent_Pill'] = np.round(tmp['Pill']/tmp['GrandTotal']*100, 2)
        tmp['Percent_Condom'] = np.round(tmp['Condom']/tmp['GrandTotal']*100, 2)
        tmp['Percent_Injectable'] = np.round(tmp['Injectable']/tmp['GrandTotal']*100, 2)
        tmp['Percent_IUD'] = np.round(tmp['IUD']/tmp['GrandTotal']*100, 2)
        tmp['Percent_Implant'] = np.round(tmp['Implant']/tmp['GrandTotal']*100, 2)
        tmp['Percent_PerMale'] = np.round(tmp['PerMale']/tmp['GrandTotal']*100, 2)
        tmp['Percent_PerFemale'] = np.round(tmp['PerFemale']/tmp['GrandTotal']*100, 2)
        tmp['CAR'] = np.round(tmp['GrandTotal']/tmp['ECouple']*100, 2)
        tmp = tmp[['upazila', 'Percent_Pill', 'Percent_Condom', 'Percent_Injectable', 
                   'Percent_IUD', 'Percent_Implant', 'Percent_PerMale',
                   'Percent_PerFemale', 'CAR',]]
        out_dict[key] = tmp
    return out_dict

def update_geos(datad, geo_df):
    out = {}
    for key, value in datad.items():
        tmp = geo_df.merge(value, how='left', left_on='upazila_dgfp', right_on='upazila')
        out[key] = tmp
    return out


def clean_up(data_dict):
    out = {}
    for key, value in data_dict.items():
        tmp = value
        tmp['geo'] = tmp['division_geo'].str.cat(tmp['zila_geo'], sep="")
        tmp = value.drop(['upazila_dgfp', 'division_dgfp', 'upazila_full_dgfp', 'division_geo',
                          'division', 'zila_geo','zila','upazila'], axis=1)
        out[key] = tmp
    return out

## Data reading and processing 

In [9]:
files = list_files(path=DATA) 
data, log = read_arrange_files(files_list=files)

### Log data

In [10]:
log['empty'].value_counts()
log_true = log[log['empty'] == True]
display(log_true.head())

,empty,full_name_path,file_name1,file_name2,year,month,upazila,division,upazila_full
22,True,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2018-10-28-0507-05.csv,2018-10-28-0507-05,2018,10,0507,05,050507
61,True,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2018-11-28-0102-07.csv,2018-11-28-0102-07,2018,11,0102,07,070102
74,True,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2018-11-28-0405-04.csv,2018-11-28-0405-04,2018,11,0405,04,040405
134,True,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2018-12-28-0305-03.csv,2018-12-28-0305-03,2018,12,0305,03,030305
160,True,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2018-12-28-0114-01.csv,2018-12-28-0114-01,2018,12,0114,01,010114


## Missing data by years

In [11]:
print(log_true['year'].value_counts())

2018    196
2008      1
Name: year, dtype: int64


## Missing data by months

In [12]:
print(log_true['month'].value_counts())

12    66
10    65
11    65
2      1
Name: month, dtype: int64


## Missing data by divisions

In [13]:
print(log_true['division'].value_counts())

04       39
05       33
02       30
07       25
01       24
03       18
06       12
08       12
07(1)     1
03(1)     1
05(1)     1
01(1)     1
Name: division, dtype: int64


In [14]:
data_df = concat_data(ddict=data)
print(data_df.shape)

(63891, 21)


In [15]:
display(data_df.head())
data_df.columns

,thana,ECouple,Pill,Condom,Injectable,IUD,Implant,PerMale,PerFemale,Tpermanent,...,CAR,Unnamed: 12,full_name_path,file_name1,file_name2,year,month,upazila,division,upazila_full
0,Boda,51742,21788,873,8018,840,3573,3183,3480,6663,...,80.6985,NaN,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2012-9-28-0103-07.csv,2012-9-28-0103-07,2012,9,0103,07,070103
1,Panchgarh Sadar,54765,23713,1733,10544,831,1899,1891,2568,4459,...,78.8442,NaN,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2012-9-28-0103-07.csv,2012-9-28-0103-07,2012,9,0103,07,070103
2,Tetulia,26033,12321,848,3058,1024,1291,794,1605,2399,...,80.4402,NaN,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2012-9-28-0103-07.csv,2012-9-28-0103-07,2012,9,0103,07,070103
3,Atowari,28599,13166,622,2986,653,1111,2257,2119,4376,...,80.1217,NaN,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2012-9-28-0103-07.csv,2012-9-28-0103-07,2012,9,0103,07,070103
4,Debiganj,47815,19301,1178,10259,903,1109,3362,2674,6036,...,81.1168,NaN,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2012-9-28-0103-07.csv,2012-9-28-0103-07,2012,9,0103,07,070103


Index(['thana', 'ECouple', 'Pill', 'Condom', 'Injectable', 'IUD', 'Implant',
       'PerMale', 'PerFemale', 'Tpermanent', 'GrandTotal', 'CAR',
       'Unnamed: 12', 'full_name_path', 'file_name1', 'file_name2', 'year',
       'month', 'upazila', 'division', 'upazila_full'],
      dtype='object')

## Break data by years

In [16]:
datad = break_by_years(input_df=data_df, year_var='year')
datad.keys()

dict_keys(['2012', '2013', '2010', '2011', '2008', '2018', '2016', '2017', '2014', '2015', '2009'])

## Summarize data 

In [17]:
data = summarize_zilas(data_dict=datad)
data = calculate_indicators(data_dict=data)
for key, value in data.items():
    print(key)
    print(value.shape)

2012
(64, 9)
2013
(64, 9)
2010
(64, 9)
2011
(64, 9)
2008
(64, 9)
2018
(64, 9)
2016
(64, 9)
2017
(64, 9)
2014
(64, 9)
2015
(64, 9)
2009
(64, 9)


## Read geographies

In [18]:
dgfp_geo = pd.read_csv(GEO)
dgfp_geo['upazila_dgfp'] = dgfp_geo['upazila_dgfp'].astype(str).str.pad(width=4, side='left', fillchar='0')
dgfp_geo['division_dgfp'] = dgfp_geo['division_dgfp'].astype(str).str.pad(width=2, side='left', fillchar='0')
dgfp_geo['upazila_full_dgfp'] = dgfp_geo['upazila_full_dgfp'].astype(str).str.pad(width=6, side='left', fillchar='0')
dgfp_geo['division_geo'] = dgfp_geo['division_geo'].astype(str).str.pad(width=2, side='left', fillchar='0')
dgfp_geo['zila_geo'] = dgfp_geo['zila_geo'].astype(str).str.pad(width=2, side='left', fillchar='0')
dgfp_geo = dgfp_geo.drop(['thana_dgfp', 'Unnamed: 8'], axis=1)
print(dgfp_geo.shape)
dgfp_geo.head()
dgfp_geo = dgfp_geo.drop_duplicates()
print(dgfp_geo.shape)

(29260, 7)
(64, 7)


## Update geographies

In [19]:
data = update_geos(datad=data, geo_df=dgfp_geo)

## Clean up before writing data

In [20]:
data = clean_up(data_dict=data)

## Writing data

In [21]:
def write_out(data_dict, out_dir):
    out = {}
    for key, value in data_dict.items():
        var_names = ["imp11subdistr_thanaprocess" + var if var != 'geo' else var for var in value.columns]
        value.columns = var_names
        value.to_csv(path_or_buf=os.path.join(out_dir, "data_dgfp_imp11subdistr_thanaprocess_" + str(key) + ".csv"), 
                    index=False, index_label=False)
write_out(data_dict=data, out_dir=OUT)